In [1]:
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.6.0
[]


In [3]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable
import segmentation_models as sm

sm.set_framework('tf.keras')
sm.framework()

model = Unet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=32, activation='softmax', input_shape=(None, None, 3))
model.summary()

# para quitar capas
# func_model._layers.pop(1)func_model._layers.pop(1)

Segmentation Models: using `keras` framework.
85532672/85521592 [==============================] - 11s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, None, None,  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
import numpy as np

# x1 = np.load('dataset_nyuv2/x1.npy',mmap_mode='r')
# x2 = np.load('dataset_nyuv2/x2.npy',mmap_mode='r')
# y = np.load('dataset_nyuv2/y.npy',mmap_mode='r')
# y = np.load('dataset_nyuv2/y_prepared_32.npy',mmap_mode='r')
# print(y.shape)
# y = np.reshape(y, (1449, 480, 640, 33), order='F')

x = np.load('dataset_sunrgbd/x1.npy',mmap_mode='r')
y = np.load('dataset_sunrgbd/y_prepared.npy',mmap_mode='r')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_sunrgbd/y_prepared.npy'

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [6]:
train_datagen.fit(X_train)

train = train_datagen.flow(X_train, y_train, batch_size=16, subset='training')
validation = train_datagen.flow(X_train, y_train, batch_size=8, subset='validation')

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_best_weights.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', min_delta=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_dice_coef", 
                      mode="max", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
model.compile(loss="binary_crossentropy", optimizer= "Adam", metrics =["accuracy"])

# Para utilizar el 20% para validación vamos a poner que realizen un split de 0,2
history = model.fit(train, y, epochs=30, verbose=1,
                    batch_size=8, validation_data=validation, callbacks=callbacks_list)

Epoch 1/30
145/145 [==============================] - 33s 189ms/step - loss: 0.0440 - accuracy: 0.0950 - val_loss: 0.0463 - val_accuracy: 0.0998
Epoch 2/30
145/145 [==============================] - 21s 146ms/step - loss: 0.0435 - accuracy: 0.0982 - val_loss: 0.0454 - val_accuracy: 0.1021
Epoch 3/30
145/145 [==============================] - 22s 151ms/step - loss: 0.0429 - accuracy: 0.1025 - val_loss: 0.0456 - val_accuracy: 0.0958
Epoch 4/30
145/145 [==============================] - 21s 147ms/step - loss: 0.0425 - accuracy: 0.1044 - val_loss: 0.0445 - val_accuracy: 0.1088
Epoch 5/30
145/145 [==============================] - 22s 149ms/step - loss: 0.0422 - accuracy: 0.1063 - val_loss: 0.0441 - val_accuracy: 0.1075
Epoch 6/30
145/145 [==============================] - 22s 149ms/step - loss: 0.0417 - accuracy: 0.1091 - val_loss: 0.0445 - val_accuracy: 0.1078
Epoch 7/30
145/145 [==============================] - 22s 155ms/step - loss: 0.0414 - accuracy: 0.1106 - val_loss: 0.0449 - val_ac

In [ ]:
# Para pasar la segmentacion a imagen
np.argmax(a, axis=1)